Testing modules to import and use MACE, based on the Colab by F. Mambretti: https://github.com/francescomambretti/atomsim_energy_mlp

If you haven't installed Ovito already, run `mamba install --strict-channel-priority -c https://conda.ovito.org -c conda-forge ovito=3.11.1` in the terminal (doesn't work from the notebook apparently).

### Test import ASE+MACE

In [ ]:
# Import for dynamics with ASE
import ase
from ase.constraints import FixAtoms
from ase.md.langevin import Langevin
from ase.md.nvtberendsen import NVTBerendsen
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase import units
from ase import io
from ase.calculators.mixing import SumCalculator
from ase.visualize import view

In [ ]:
# Import for postprocessing & utils
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os

In [ ]:
# Testing the import of Ovito
from ovito.io import import_file, export_file
from ovito.modifiers import CoordinationAnalysisModifier, TimeAveragingModifier, SelectTypeModifier

In [ ]:
# Testing the import of MACE
from mace.calculators import mace_mp
calc = mace_mp(model="medium", dispersion=False, device='cpu')

## Foundation model to run NVT simulation

From: https://mace-docs.readthedocs.io/en/latest/examples/foundation_examples.html

Links to the tutorials:
- https://colab.research.google.com/drive/1ZrTuTvavXiCxTFyjBV4GqlARxgFwYAtX#scrollTo=UIdFlyFDw9cy
- https://colab.research.google.com/drive/1oCSVfMhWrqHTeHbKgUSQN9hTKxLzoNyb
- https://colab.research.google.com/drive/1AlfjQETV_jZ0JQnV5M3FGwAM2SGCl2aU

Other useful links:
- https://mace-docs.readthedocs.io/en/latest/guide/lammps.html
- https://docs.lammps.org/Build_extras.html#kokkos
- https://github.com/ACEsuit/mace-mp/releases/tag/mace_mp_0

In [1]:
from mace.calculators import mace_mp
from ase import build
from ase.md import Langevin
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase import units

/home/michele/mambaforge/envs/workflow-hea/lib/python3.10/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_indices = t

In [5]:
macemp = mace_mp()
#macemp = mace_mp(model="medium", dispersion=False, device='cpu')
#macemp = mace_mp(model="large") # return a larger model
#macemp = mace_mp(model="https://tinyurl.com/y7uhwpje") # downlaod the model at the given url
#macemp = mace_mp(dispersion=True) # return a model with D3 dispersion correction
atoms = build.molecule('H2O')
atoms.calc = macemp

Using Materials Project MACE for MACECalculator with /home/michele/.cache/mace/20231203mace128L1_epoch199model
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.


/home/michele/mambaforge/envs/workflow-hea/lib/python3.10/site-packages/mace/calculators/mace.py:130: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, 

RuntimeError: don't know how to restore data location of torch.storage.UntypedStorage (tagged with gpu)

In [3]:
# Initialize velocities.
T_init = 300  # Initial temperature in K
MaxwellBoltzmannDistribution(atoms, T_init * units.kB)

/home/michele/mambaforge/envs/workflow-hea/lib/python3.10/site-packages/ase/md/md.py:53: FutureWarning: Specify the temperature in K using the 'temperature_K' argument
  warnings.warn(FutureWarning(w))


In [4]:
# Set up the Langevin dynamics engine for NVT ensemble.
dyn = Langevin(atoms, 0.5 * units.fs, T_init * units.kB, 0.001)
n_steps = 200 # Number of steps to run
dyn.run(n_steps)

True